In [1]:
import json
import glob
import os
import numpy as np
import sys
from scipy.sparse import csr_matrix

In [2]:
def cardlist():
    #needs JSON
    with open('AtomicCards.json', encoding="utf8") as json_file:
        data = json.load(json_file)
    #make a list of all cardnames
    a = []
    for p in data["data"]: 
       a.append(p)
    return(a)  
  
        

In [20]:
def cardlist_sub():
    #makes a dictionary of alternate cardnames names that show up
    d={}
    for name in cardlist():
        if "//" in name:
            #accounts for formats where only the front face is given
            short=name[:name.find("//")-1]
            d[short] = name
            #accounts for formats where one slash is used
            one_slash=name[:name.find("//")]+name[name.find("//")+1:]
            d[one_slash] = name
    #accounts for random hiccups in Decklist import       
    d["Lim-Dul's Vault"] = "Lim-Dûl's Vault"
    d["Sire Of Insanity"] = "Sire of Insanity"
    return d

In [21]:
def decklist_to_dict(s="./decklists/Gargos Chomp-1.txt"):
    d = {}
    with open(s) as f:
        for line in f:
            if not line.isspace():
                (val, key) = line.strip().split(maxsplit=1)
                d[key] = int(val)
    return(d)

In [22]:
def make_number_list(d, filename):
    dl= d
    cl= cardlist()
    cl_sub= cardlist_sub()
    sum=0

    a=np.zeros(shape=(len(cardlist())),dtype=np.int8)
    
    for i in dl:
        if i in cl:
            a[cl.index(i)]=dl[i]
            sum=sum+dl[i]
        elif i in cl_sub:
            a[cl.index(cl_sub[i])]=dl[i]
            sum=sum+dl[i]
        else:    
            print("Cardname '" + i + "' in Decklist: '"+filename+"' unknown")       
#check if list is complete
    
    if not sum==100:
        print("Decksize of " + filename + " is " + str(sum))
    
    return a

In [23]:
def count_decklists(s="./decklists"):
   #needs glob and os
    path = s
    i=0
    for filename in glob.glob(os.path.join(path, '*.txt')):
        i=i+1
    return i

In [24]:
def decklists_to_array(s="./decklists"):
    #needs glob, os and np
    path = s
    boards=["SIDEBOARD:","Sideboard:","Sideboard","MAYBEBOARD","Maybeboard:","Maybeboard", "Bench", "In","// Outside the Game", "To Add", "Dream-lands", "Cuts", "Cards to Be Bought"]
    #define array with size appropriate to data
    big = np.zeros(shape=(count_decklists(s),len(cardlist())+1),dtype=np.int8)
    i=-1
    for filename in glob.glob(os.path.join(path, '*.txt')):
        #make dictionary
        with open(os.path.join(os.getcwd(), filename), 'r', encoding="utf8") as f: # open in readonly mode
            d = {}
            for line in f:
                try:
                    #check for empty lines
                    if not line.isspace():
                        #check for sideboard
                        if line.strip() in boards: 
                            break
                        else:
                            (val, key) = line.strip().split(maxsplit=1)
                            if not key in d:
                                d[key] = int(val)
                            else:
                                d[key] = d[key]+int(val)
                except:
                    print(line+" in "+filename+" is not the right format")
        #set the appropriate line in array to the number list. Last spot stays reserved for Decktype            
        i=i+1
        big[i][:-1]=make_number_list(d, filename)
    return(big)

In [25]:
def make_matrix(path,pls):
    #needs powerlevel dictionary with values
    le=len(cardlist())
    matrix=np.zeros((0,le+1),"int8")
    total_files=0
    for foldername in glob.iglob(os.path.join(path, '*')):
         for pl in pls:
            if pl in foldername:
                total_files=total_files+count_decklists(foldername)
    matrix=np.zeros((total_files,len(cardlist())+1),"int8")          
    old=0
    for i,foldername in enumerate(glob.iglob(os.path.join(path, '*'))):
         for pl in pls:
            if pl in foldername:
                new = old + count_decklists(foldername)
                matrix[old:new]=decklists_to_array(foldername) 
                matrix[old:new,le:]= pls[pl]
                old = new
    return matrix

In [47]:
def merge_matrix(a,b):
    if not a.shape[1]==b.shape[1]:
        raise NameError("Matrices not compatible, "+str(a.shape[1])+"not equal"+str(b.shape[1]))
    c=np.zeros((a.shape[0]+b.shape[0],a.shape[1]),"int8")
    c[:a.shape[0]]=a
    c[a.shape[0]:]=b
    return c

In [26]:
p={"BC":1,"MID":2,"HIGH":3,"COMP":4}
p_mod={"BC":1,"HIGH":3,"COMP":4}

In [29]:
%%time
m=make_matrix("./decklists/",p_mod)


Decksize of ./decklists\COMP\16-11-17-troll-deck.txt is 99
Decksize of ./decklists\COMP\scions-breakfast-dragon-loops-cedh.txt is 103
Wall time: 8min 35s


In [30]:

print(m.shape)
print(np.sum(m[:,:-1]))

(84, 21455)
8402


In [35]:
np.save("matrix2.npy",m)

In [48]:
a=np.load("matrix.npy")
b=np.load("matrix2.npy")
c=merge_matrix(a,b)
c.shape

(412, 21455)

In [49]:
np.save("matrixc.npy",c)

In [51]:
name="abc // cde"
print(name[:name.find("//")]+name[name.find("//")+1:])


abc / cde
